#  Training YOLO on Fire Detection for outdoor Scenario

## Overview
In this notebook, we train a **YOLOv8** model on a custom dataset designed for detecting **fire scenarios** in **outdoor images**. This dataset represents one of the specific scenarios for our **Mixture of Experts (MoE)** model, where each expert specializes in a different scenario (e.g., fire detection in outdoor, indoor, satellite, or far-field environments).

### Key Steps:
- **Dataset**: The model is trained using a **outdoor fire detection dataset**, which consists of images with fire-related features captured in street view or general pictures in the outdoor.
- **YOLOv8 Training**: The YOLOv8 model is fine-tuned on this dataset, learning to identify fire-related objects.
- **Scenario Expert**: This trained model acts as an expert specifically for detecting fire in outdoor images and is part of a broader MoE-based approach for multi-scenario detection.



##  Data Loading & Preprocessing


In [1]:
import os
import random
import shutil

# Set paths
test_images_dir = 'test/images'
test_labels_dir = 'test/labels'
val_images_dir = 'valid/images'
val_labels_dir = 'valid/labels'

# Create destination directories if they don't exist
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Set random seed for reproducibility
random.seed(42)

# List of test image filenames
test_images = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg') or f.endswith('.png')]

# Randomly select 300 for validation
val_images = random.sample(test_images, 300)

# Move selected images and their labels to `valid/`
for img_file in val_images:
    # Corresponding label file (YOLO format: same name with .txt extension)
    label_file = os.path.splitext(img_file)[0] + '.txt'
    
    # Move image
    shutil.move(os.path.join(test_images_dir, img_file), os.path.join(val_images_dir, img_file))
    
    # Move label (only if it exists)
    label_path = os.path.join(test_labels_dir, label_file)
    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(val_labels_dir, label_file))

print(f"Moved {len(val_images)} images and their labels to the validation set.")


Moved 300 images and their labels to the validation set.


##  Training YOLOv8 on Outdoor Fire Detection

We utilize the **YOLOv8** object detection model to train on a custom dataset defined in `outdoor.yaml`. The configuration for training is as follows:

- 🧠 **Model**: `yolov8n.pt` (Nano variant — optimized for speed and prototyping)
- 📁 **Dataset**: Defined in `outdoor.yaml`
- 🖼️ **Image Size**: 640×640
- 🔁 **Epochs**: 100
- 📦 **Batch Size**: 16
- 💻 **Device**: GPU (device 0)




In [2]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0))

GPU available: True
Current device: NVIDIA GeForce RTX 3070


In [4]:
from ultralytics import YOLO

# Load a YOLOv8 model (choose from yolov8n, yolov8s, yolov8m, etc.)
model = YOLO("yolov8n.pt")  # you can replace with 'yolov8s.pt' for better accuracy

# Train the model
model.train(
    data="outdoor.yaml",     # Path to your custom dataset YAML
    epochs=100,                # Number of training epochs
    imgsz=640,                 # Image size
    batch=16,                  # Batch size (adjust based on your GPU memory)
    device = 0
)


Ultralytics 8.3.107  Python-3.11.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=outdoor.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None

100%|██████████| 5.35M/5.35M [00:01<00:00, 2.86MB/s]


AMP: checks passed 


train: Scanning C:\Users\pc\Desktop\DL\Project\Outdoor\train\labels... 2621 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2621/2621 [00:02<00:00, 885.37it/s]


train: New cache created: C:\Users\pc\Desktop\DL\Project\Outdoor\train\labels.cache


...

                   all        300        774      0.747      0.664      0.729      0.411



100 epochs completed in 0.535 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.107  Python-3.11.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  6.07it/s]


                   all        300        774      0.741      0.669      0.729      0.413
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024B02593750>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

##  Inference on Outdoor Test Images using Trained YOLOv8 Model

We perform inference using the **best trained YOLOv8 model** to evaluate its performance on the **test set** of outdoor fire scenarios.

- 🎯 **Model Weights**: `best.pt` from the training run
- 💾 **Output**: Predictions will be saved automatically by YOLO

This step visually validates how well the model generalizes to unseen indoor fire detection cases.


In [5]:
from ultralytics import YOLO

# Load your trained weights
model = YOLO("runs/detect/train/weights/best.pt")


In [6]:
results = model.predict(source="C:/Users/pc/Desktop/DL/Project/Outdoor/test/images", save=True)



image 1/363 C:\Users\pc\Desktop\DL\Project\Outdoor\test\images\084d8e970b8d456a_jpg.rf.00a44feed93ffa5716c81d5cdb3f8f57.jpg: 480x640 1 Fire, 74.3ms
image 2/363 C:\Users\pc\Desktop\DL\Project\Outdoor\test\images\0d1db917073dd5b8_jpg.rf.0e31edc3f90d9ac9b89278c7a8e50a9f.jpg: 640x512 1 Fire, 43.9ms
image 3/363 C:\Users\pc\Desktop\DL\Project\Outdoor\test\images\0d4b3896f938d981_jpg.rf.1b3d0ee1ae27514c3abe54175453539d.jpg: 448x640 1 Fire, 53.8ms
image 4/363 C:\Users\pc\Desktop\DL\Project\Outdoor\test\images\10-image_jpg.rf.6eb3e1f0a10cb894cecad1bc7fef93f4.jpg: 416x640 3 Fires, 45.9ms
image 5/363 C:\Users\pc\Desktop\DL\Project\Outdoor\test\images\1019_jpg.rf.f897077937ec7b1b3c8f2881009423f7.jpg: 384x640 4 Fires, 41.2ms
image 6/363 C:\Users\pc\Desktop\DL\Project\Outdoor\test\images\1072_jpg.rf.c88e346bc0652df15bac08585204d115.jpg: 448x640 1 Fire, 9.1ms
image 7/363 C:\Users\pc\Desktop\DL\Project\Outdoor\test\images\1079_jpg.rf.6ebb45fe41c736d70711dfa5d44876ec.jpg: 480x640 4 Fires, 4.8ms
image 